In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.utils import shuffle
%matplotlib inline

In [2]:
df = pd.read_csv("Data/openpowerlifting.csv")

C:\Users\tvaino\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Remove unnecessary columns
da = df#df[['Name', 'Sex', 'Event', 'Equipment', 'Age', 'Wilks', 'Tested', 'Date', 'BodyweightKg']]



In [4]:
# Changing column tested nan rows to no
da['Tested'] = da['Tested'].replace(np.nan,0)
da['Tested'] = da['Tested'].replace('Yes',1)
da['Sex'] = da['Sex'].replace('M',0)
da['Sex'] = da['Sex'].replace('F',1)

In [5]:
# Rows where event is not SBD will be dropped
da = da[da.Event=='SBD']

In [6]:
#Drop rows where age is nan
da = da.dropna(how='any', subset=['Name','Sex', 'Equipment', 'Age', 'Wilks', 'Date', 'BodyweightKg'])

In [7]:
da['Equipment'] = da['Equipment'].replace('Raw',0)
da['Equipment'] = da['Equipment'].replace('Single-ply',1)
da = da[(da['Equipment']==1) | (da['Equipment']==0)]

In [8]:
#Adding column with approximate date of birth
da['BirthYear'] = da.apply(lambda row: int(str(row.Date)[:4]) - (row.Age), axis = 1)

In [9]:
#Sort by date and names
da = da.sort_values(['Name','Date'])

In [10]:
da['PersonID'] = 0
da['PreviousComps'] = 0
p_id = 0
last_name = ""
last_birth_year = 0
last_sex = 2
competition_rate = 0

for i in range(len(da)):
    row = da.iloc[i]
    new_name = row['Name']
    new_birth_year = row['BirthYear']
    new_sex = row['Sex']
    if new_name == last_name:
        if new_sex == last_sex and abs(new_birth_year-last_birth_year)<=1:
            #print(new_name)
            competition_rate += 1
            da['PersonID'].iloc[i] = p_id
            da['PreviousComps'].iloc[i] = competition_rate
        else:
            competition_rate = 1
            last_name = new_name
            last_birth_year = new_birth_year
            last_sex = new_sex
            p_id+=1
            da['PersonID'].iloc[i] = p_id
            da['PreviousComps'].iloc[i] = competition_rate
    else:
        competition_rate = 1
        last_name = new_name
        last_birth_year = new_birth_year
        last_sex = new_sex
        p_id+=1
        da['PersonID'].iloc[i] = p_id
        da['PreviousComps'].iloc[i] = competition_rate
da.head(10)
        
    

C:\Users\tvaino\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName,BirthYear,PersonID,PreviousComps
913534,A Abduzhabarov,0,SBD,1,23.5,24-34,Open,74.00,74,140.0,...,1,NaN,KPF,2012-03-26,Kazakhstan,NaN,Republic Kazakhstan Powerlifting Cup,1988.5,1,1
1369333,A Ajeesha,1,SBD,0,16.5,16-17,Sub-Juniors,71.10,72,102.5,...,1,India,AsianPF,2017-12-04,India,NaN,Asian Classic Powerlifting Championships,2000.5,2,1
1371420,A Ashwin,0,SBD,1,16.5,16-17,Sub-Juniors,81.70,83,150.0,...,1,India,AsianPF,2012-05-01,India,NaN,Asian Powerlifting Championships,1995.5,3,1
943185,A Bauer,0,SBD,1,20.5,20-23,Juniors,51.70,52,NaN,...,1,NaN,FPR,1999-03-25,Russia,NaN,Russian Juniors Powerlifting Championships,1978.5,4,1
1370862,A Dilmuradov,0,SBD,1,65.5,65-69,Masters 3,59.90,60,80.0,...,1,NaN,AsianPF,2004-04-28,Uzbekistan,NaN,Asian Powerlifting Championships,1938.5,5,1
1373689,A Dilymuradov,0,SBD,1,69.5,65-69,Masters 3,59.05,60,-80.0,...,1,Uzbekistan,AsianPF,2008-05-04,Uzbekistan,NaN,Asian Powerlifting Championships,1938.5,6,1
943315,A Dzhaman,1,SBD,1,16.5,16-17,Juniors,47.65,48,NaN,...,1,NaN,FPR,1999-03-25,Russia,NaN,Russian Juniors Powerlifting Championships,1982.5,7,1
774343,A DziewiczKiewicz,1,SBD,1,21.5,20-23,Juniors,73.40,75,NaN,...,1,Poland,EPF,2001-06-14,France,NaN,European Junior Powerlifitng Championships,1979.5,8,1
944759,A Eginov,0,SBD,1,24.5,24-34,Open,107.90,110,NaN,...,1,NaN,FPR,2000-09-27,Russia,NaN,Russian Powerlifting Cup,1975.5,9,1
1029329,A Ernandos-Ortega,0,SBD,1,17.5,18-19,Sub-Juniors,74.40,75,NaN,...,1,Russia,IPF,2002-09-25,Taiwan,NaN,World Sub-Juniors Powerlifting Championships,1984.5,10,1


In [11]:
da.to_csv(r'data/bigdata.csv')

In [12]:
da.columns

Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass', 'Division',
       'BodyweightKg', 'WeightClassKg', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg',
       'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg',
       'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg',
       'Deadlift4Kg', 'Best3DeadliftKg', 'TotalKg', 'Place', 'Wilks',
       'McCulloch', 'Glossbrenner', 'IPFPoints', 'Tested', 'Country',
       'Federation', 'Date', 'MeetCountry', 'MeetState', 'MeetName',
       'BirthYear', 'PersonID', 'PreviousComps'],
      dtype='object')

In [13]:
da = pd.read_csv("data/bigdata.csv")

C:\Users\tvaino\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,32,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
da = da.reset_index()

In [15]:
da = da[da.Age>=15]

WilksRanges = [600,550,500,450,400,350,300,250,200,150,100,50,0]
da['WilksRangeCat'] = -1
for i in range(len(da)):
    row = da.iloc[i]
    for cat_i in range(len(WilksRanges)):
        if row['Wilks']>WilksRanges[cat_i]:
            da['WilksRangeCat'].iloc[i] = cat_i
            break

C:\Users\tvaino\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
da['WilksRangeCat'].value_counts()

5     88779
6     81104
4     64218
7     46915
3     37664
8     17563
2     17277
1      5986
9      3807
0      1506
10      487
11       70
12        9
Name: WilksRangeCat, dtype: int64

In [17]:
# Splitting date into four features
da['Year'] = da.apply(lambda row: int(str(row.Date)[:4]), axis = 1)
da['Month'] = da.apply(lambda row: int(str(row.Date)[5:7]), axis = 1)
da['Day'] = da.apply(lambda row: int(str(row.Date)[8:10]), axis = 1)
da['Date'] = pd.to_datetime(da['Date'])
da['Weekday'] = da['Date'].dt.dayofweek

In [18]:
# Remove 
da = da[['PreviousComps', 'Sex', 'Equipment', 'Age', 'Wilks', 'Tested', 'Year','Month','Day','Weekday', 'BodyweightKg', 'WilksRangeCat']]

In [19]:
# Splitting data into test and train data set
da = shuffle(da, random_state=2)
split_id = int(len(da)*0.7)
da_train =da.iloc[:split_id]
da_test = da.iloc[split_id:]

In [20]:
# Saving clean data to file
da_test.to_csv(r'data/test_data.csv')
da_train.to_csv(r'data/train_data.csv')